In [2]:
import pandas as pd
import re

In [3]:
mageck_mle_out = snakemake.output[0]
mageck_rra_out = snakemake.output[1]
bagel_out = snakemake.output[2]
crisphiermix_out = snakemake.output[3]

In [4]:
files = {}
for i in range(0, len(snakemake.input)):
    m = re.search("^results\/development\/(\w+)\/", snakemake.input[i])
    if m:
        files[m.group(1)] = snakemake.input[i]

files

In [5]:
token = snakemake.wildcards[0]
treatment = snakemake.wildcards[1]
control = snakemake.wildcards[2]

print("Token:", token)
print("Treatment:", treatment)
print("Control:", control)

In [6]:
mageck_mle_file = files['MAGeCK_MLE']
mageck_mle_data = pd.read_csv(mageck_mle_file, sep="\t")
mageck_mle_data

In [7]:
beta = treatment + "|beta"
z = treatment + "|z"
fdr = treatment + "|fdr"

print("Beta:", beta)
print("Z:", z)
print("FDR:", fdr)

In [8]:
mageck_mle_neg = mageck_mle_data.loc[(mageck_mle_data[fdr]<0.05) & (mageck_mle_data[beta]<0)][['Gene', 'sgRNA', beta, z, fdr]]
mageck_mle_neg

In [9]:
mageck_mle_neg_genes = list(mageck_mle_neg['Gene'].values)
len(mageck_mle_neg_genes)

In [10]:
mageck_mle_pos = mageck_mle_data.loc[(mageck_mle_data[fdr]<0.05) & (mageck_mle_data[beta]>0)][['Gene', 'sgRNA', beta, z, fdr]]
mageck_mle_pos

In [11]:
mageck_mle_pos_genes = list(mageck_mle_pos['Gene'].values)
len(mageck_mle_pos_genes)

In [12]:
mageck_rra_file = files['MAGeCK_RRA']
mageck_rra_data = pd.read_csv(mageck_rra_file, sep="\t")
mageck_rra_data

In [13]:
mageck_rra_pos = mageck_rra_data.loc[(mageck_rra_data["pos|fdr"]<0.05) & (mageck_rra_data["neg|lfc"]>0)]
mageck_rra_pos

In [14]:
mageck_rra_pos_genes = list(mageck_rra_pos['id'].values)
len(mageck_rra_pos_genes)

In [15]:
mageck_rra_neg = mageck_rra_data.loc[(mageck_rra_data["neg|fdr"]<0.05) & (mageck_rra_data["neg|lfc"]<0)]
mageck_rra_neg

In [16]:
mageck_rra_neg_genes = list(mageck_rra_neg['id'].values)
len(mageck_rra_neg_genes)

In [17]:
bagel_file = files['BAGEL']
bagel_data = pd.read_csv(bagel_file, sep="\t")
bagel_data

In [18]:
bagel_neg = bagel_data.loc[(bagel_data['BF']>0)]
bagel_neg

In [19]:
bagel_neg_genes = list(bagel_neg['GENE'].values)
len(bagel_neg_genes)

In [20]:
chrm_file = files['CRISPhieRmix']
chrm_data = pd.read_csv(chrm_file, sep=",")
chrm_data

In [21]:
chrm_res = chrm_data.loc[(chrm_data['locfdr']<0.05)]
chrm_res

In [22]:
chrm_genes = list(chrm_res['gene'].values)
len(chrm_genes)

In [28]:
with open(mageck_mle_out, 'w') as f:
    for gene in mageck_mle_neg_genes:
        f.write("%s\n" % gene)
        
with open(mageck_rra_out, 'w') as f:
    for gene in mageck_rra_neg_genes:
        f.write("%s\n" % gene)

with open(crisphiermix_out, 'w') as f:
    for gene in chrm_genes:
        f.write("%s\n" % gene)

with open(bagel_out, 'w') as f:
    for gene in bagel_neg_genes:
        f.write("%s\n" % gene)

In [30]:
def write_output_file(genes, output):
    with open(output, 'w') as f:
        for gene in genes:
            f.write("%s\n" % gene)

            
write_output_file()          

In [59]:
# print(bagel_neg)
mageck_rra_pos = mageck_rra_data.loc[(mageck_rra_data["pos|fdr"]<0.05) & (mageck_rra_data["pos|lfc"]>0)]
mageck_mle_pos = mageck_mle_data.loc[(mageck_mle_data[fdr]<0.05) & (mageck_mle_data[beta]>0)][['Gene', 'sgRNA', beta, z, fdr]]

a = set(bagel_neg['GENE'].values)
b = set(mageck_mle_neg['Gene'].values)
c = set(mageck_rra_neg['id'].values)
d = set(chrm_res['gene'].values)


In [60]:
# BAGEL - MLE
len(set(a) & set(b))

In [61]:
# BAGEL - RRA
len(set(a) & set(c))

In [62]:
# RRA - MLE
len(set(c) & set(b))

In [63]:
# BAGEL - CRISPhieRmix
len(set(a) & set(d))

In [66]:
# MLE - CRISPhieRmix
print(len(set(b) & set(d)))
# set(b) & set(d)

In [67]:
# RRA - CRISPhieRmix
print(len(set(c) & set(d)))
# set(c) & set(d)